<a href="https://colab.research.google.com/github/nitinsb/TensorFlow/blob/main/colabs/CNNs_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks (CNNs)

### Import TensorFlow

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu
import tensorflow as tf
import datetime, os

from tensorflow.keras import datasets, layers, models

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


CNNs are used extensively in Computer Vision.  They are also used for modeling temporal data.


The key idea in CNNs is to **capture local patterns in the data through a convolution operation and then to downsample the resulting output with the pooling layer**.   CNNs employ a series of convolution-pooling layers.


Let's understand these operations in the context of image classification application.  In this case CNNs take an image as an input and predicts the corresponding digit.  The image is encoded as a 3d tensor with axes corresponding to height, width and depth of an image.  


### Download and prepare the MNIST dataset

In the case of MNIST data, we are dealing with grey scale images and hence number of depth is 1 while height and width are both 28.  We will download and preprocess MNIST handwritten digit recognition dataset just as before except for a small change.   Here we reshape each image into a 3D tensor so that the input becomes a 4D tensor.  This is done to satisfy the input requirements of the CNNs.


In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

print ("Before reshaping:")
print ("=================")
print ("Number of axes in train_images: ", train_images.ndim)
print ("Number of axes in test_images: ", test_images.ndim)
print ("Shape of train_images: ", train_images.shape)
print ("Shape of test_images: ", test_images.shape)
print ()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

print ("After reshaping:")
print ("=================")
print ("Number of axes in train_images: ", train_images.ndim)
print ("Number of axes in test_images: ", test_images.ndim)
print ("Shape of train_images: ", train_images.shape)
print ("Shape of test_images: ", test_images.shape)
print ()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

11490434/11490434 [==============================] - 0s 0us/step
Before reshaping:
Number of axes in train_images:  3
Number of axes in test_images:  3
Shape of train_images:  (60000, 28, 28)
Shape of test_images:  (10000, 28, 28)

After reshaping:
Number of axes in train_images:  4
Number of axes in test_images:  4
Shape of train_images:  (60000, 28, 28, 1)
Shape of test_images:  (10000, 28, 28, 1)



We print the number of axes and shape of train and test image tensors before and after the reshaping.  You can observe that the reshaping operation converts the input 3D tensor into the 4D tensor.   An additional axes is added in each image.


### Create the convolutional base

In [3]:
model = models.Sequential()

The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to color channels, MNIST has one (because the images are grayscale), whereas a color image has three (R,G,B). In this example, we will configure our CNN to process inputs of shape (28, 28, 1), which is the format of MNIST images. We do this by passing the argument input_shape to our first layer.

In [6]:

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))


In [7]:
model.add(layers.MaxPooling2D((2, 2))) # for downsampling


In [8]:
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

In [9]:

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Let display the architecture of our model so far.



In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55744 (217.75 KB)
Trainable params: 55744 

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as we go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, we can afford (computationally) to add more output channels in each Conv2D layer.

### Add Dense layers on top
To complete our model, we will feed the last output tensor from the convolutional base (of shape (3, 3, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, we will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. MNIST has 10 output classes, so we use a final Dense layer with 10 outputs and a softmax activation.

In [12]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

 Here's the complete architecture of our model.

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

As you can see, our (3, 3, 64) outputs were flattened into vectors of shape (576) before going through two Dense layers.

### Compile and train the model

In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 12s 4ms/step - loss: 0.1364 - accuracy: 0.9578
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0452 - accuracy: 0.9861
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0327 - accuracy: 0.9898
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0238 - accuracy: 0.9925
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0195 - accuracy: 0.9936


### Evaluate the model

In [15]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0276 - accuracy: 0.9920
0.9919999837875366


As you can see, our simple CNN has achieved a test accuracy of over 99%.

### Classification with Feedforward Neural Networks

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [17]:
model_dnn = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model_dnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model_dnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model_dnn.fit(x_train, y_train, epochs=5)
model_dnn.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2925 - accuracy: 0.9152
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1403 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1059 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0883 - accuracy: 0.9728
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0761 - accuracy: 0.9754


[0.07610346376895905, 0.9753999710083008]